In [2]:
import open3d as o3d
import pyrealsense2 as rs
import time

In [ ]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848, 480, rs.format.rgb8, 30)

pipeline.start(config)

# skip some frames
for x in range(30):
    pipeline.wait_for_frames()

align_to = rs.stream.color
align = rs.align(align_to)
add = False

# Filter
decimation = rs.decimation_filter()
spatial = rs.spatial_filter()
temporal = rs.temporal_filter()

try:
    alive = True
    vis = o3d.visualization.VisualizerWithKeyCallback()
    def key_action_callback(vis, action, mods):
        nonlocal alive
        if action == 0:
            alive = False
        return True
    
    vis.register_key_action_callback(ord('q'), key_action_callback)
    vis.register_key_action_callback(ord('Q'), key_action_callback)
    vis.create_window('ShowTime', width=1280, height=720)
    pcd = o3d.geometry.PointCloud()
    
    while alive:
        t0 = time.time()
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        profile = aligned_frames.get_profile()
        color = aligned_frames.get_color_frame()
        depth = aligned_frames.get_depth_frame()
        depth = decimation.process(depth)
        depth = spatial.process(depth)
        depth = temporal.process(depth)
        t1 = time.time()
        if not color or not depth:
            continue
        
        d_img = np.asanyarray(depth.get_data())
        d_img = nonZeroSubsampling(d_img, kernel_size=2, method='mean')
        img = np.asanyarray(color.get_data())
        img = cv2.resize(img, (424, 240))
        
        d_img = o3d.geometry.Image(d_img)
        img = o3d.geometry.Image(img)
        
        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(img, d_img, depth_trunc=5.0, convert_rgb_to_intensity=False)
        intrinsics = profile.as_video_stream_profile().get_intrinsics()
        pinhole_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(intrinsics.width, intrinsics.height, intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy)
        pc = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, pinhole_camera_intrinsic)
        pc.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
        pcd.points = pc.points
        pcd.colors = pc.colors
        
        if not add:
            vis.add_geometry(pcd)
            add = True
            
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        print('[FPS]: %.2f' % (1/(t1 - t0)))
finally:
    pipeline.stop()
    vis.destroy_window()